In [1]:
import pandas as pd #modules
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import pathlib
from pathlib import Path
from datetime import datetime, timedelta
import itertools
import string
from pptx import Presentation
from pptx.util import Inches, Pt
import reports_ as rp

# Load shift hours

In [2]:
def find_files_by_format(root_folder, file_format):
    """
    Searches for all files with a specific format in a folder and its subfolders.

    Parameters:
    root_folder (str): The root folder to start the search.
    file_format (str): The file extension to search for (e.g., '.pptx', '.txt').

    Returns:
    list: A list of absolute paths to all matching files.
    """
    matching_files = []

    # Traverse directories recursively
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            # Check if the file has the specified format
            if file.lower().endswith(file_format.lower()):  # Case-insensitive check
                # Append the full path of the file
                matching_files.append(os.path.join(root, file))

    return matching_files

In [3]:
def operate_mercyful_times(hour,operand): #function to apply modifications on shift hours
    from datetime import datetime, timedelta
    delta_time_mercy=timedelta(minutes=0)
    mercy_flag=False #bool to activate mercy mode
    if mercy_flag:
        delta_time_mercy=timedelta(minutes=10) #time delta to substract and add to initial and final hours
    if operand=="+" and mercy_flag: #to add to final hour
        result_hour=eval("datetime.strptime(hour, '%H:%M:%S'){}delta_time_mercy".format(operand))
        result_hour=result_hour.time()
        return result_hour
    elif operand=="-" and mercy_flag: #to substract to initial hour
        result_hour=eval("datetime.strptime(hour, '%H:%M:%S'){}delta_time_mercy".format(operand))
        result_hour=result_hour.time()
        return result_hour
    else:
        return eval("datetime.strptime(hour, '%H:%M:%S').time()")

In [4]:
str___splitter=lambda x: x.split("_")

# Function to find cloud (OneDrive) folder

In [5]:
def find_reports_in_onedrive():
    """
    Scans the subfolders under the current user's OneDrive folder (including variations like 'OneDrive - Company Name')
    and returns the paths of all folders with the prefix 'reports'.

    Returns:
        list: A list of full paths to folders starting with 'reports', or an empty list if none are found.
    """
    # Get the base path to the user's home directory
    user_home = os.path.expanduser("~")

    # Find the OneDrive folder (handles variations like "OneDrive - Company Name")
    onedrive_folder = None
    for folder in os.listdir(user_home):
        if folder.startswith("OneDrive -"):
            onedrive_folder = os.path.join(user_home, folder)
            break

    if not onedrive_folder:
        raise FileNotFoundError("OneDrive folder not found for the current user.")

    # Search for folders with the prefix 'reports' in the OneDrive directory
    report_folders = []
    for root, dirs, files in os.walk(onedrive_folder):
        for dir_name in dirs:
            if dir_name.lower().startswith("reports"):
                report_folders.append(os.path.join(root, dir_name))

    return report_folders

In [6]:
reports_paths=find_reports_in_onedrive()
reports_paths

['C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_control_variables_refinacion',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_cuchillas',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_eventos',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_humidity_measures',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_ops_pendings',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_pre-usos_diligenciamiento_no_cumple',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_quality_deviations',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_refination_deviations',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_visualizacion_data_produccion',
 'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\control_room\\reports_causa_rechazos']

In [7]:
str_folder_searcher="reports_pre-usos_diligenciamiento_no_cumple"
for report_path in reports_paths:
    if str_folder_searcher in report_path:
        path=Path(reports_paths[reports_paths.index(report_path)])
path=Path.joinpath(path,r"source_and_return_data")
path

WindowsPath('C:/Users/jdbustamante/OneDrive - Duratex SA/reports_pre-usos_diligenciamiento_no_cumple/source_and_return_data')

In [8]:
os.chdir(path) #change working directory to respective reports folder
os.getcwd()

'C:\\Users\\jdbustamante\\OneDrive - Duratex SA\\reports_pre-usos_diligenciamiento_no_cumple\\source_and_return_data'

In [9]:
shifts_hours=pd.read_csv(r".\shifts_hours.txt",delimiter=",")
shifts_hours_cols=list(shifts_hours.columns)
shifts_hours.loc[:,shifts_hours_cols[0]]=shifts_hours.loc[:,shifts_hours_cols[0]].apply(lambda x: operate_mercyful_times(hour=x,operand="-"))
shifts_hours.loc[:,shifts_hours_cols[1]]=shifts_hours.loc[:,shifts_hours_cols[1]].apply(lambda x: operate_mercyful_times(hour=x,operand="+"))
shifts_hours.loc[:,shifts_hours_cols[-1]]=shifts_hours.loc[:,shifts_hours_cols[-1]].apply(str___splitter)
shifts_hours

,initial_hour,finalitation_hour,shift,systems
0,22:00:00,06:00:00,1,"[refinación, planta térmica, formación y prens..."
1,06:00:00,14:00:00,2,"[refinación, planta térmica, formación y prens..."
2,14:00:00,22:00:00,3,"[refinación, planta térmica, formación y prens..."
3,07:00:00,16:30:00,4,[mantenimiento mecánico]


# Generation of querie to filter dates to look up completion dates

In [10]:
def get_monday_of_week():
    """
    This function, `get_monday_of_week`, returns the date  
    of the Monday in the current week based on today's date.  
    
    It first retrieves today's date using `datetime.today()`  
    and then determines the current weekday using  
    `today.weekday()`, where Monday is represented by 0.  
    
    Next, it calculates the number of days to subtract to  
    reach Monday by taking the value of `today.weekday()`  
    and subtracting that number of days using `timedelta`.  
    
    Finally, it formats the result as 'YYYY-MM-DD' and  
    returns it as the output.  
    """
    # Obtener la fecha de hoy
    today = datetime.today()
    # Calcular la diferencia de días hasta el lunes (Monday = 0)
    delta = today.weekday()  # weekday() devuelve 0 para lunes
    # Restar los días necesarios para llegar al lunes
    monday = today - timedelta(days=delta)
    # Retornar la fecha en formato YYYY-MM-DD
    return monday.date()

In [11]:
def date_limits(moment_of_week=True,day_separation=1):
    """change dates limits to filter completions 
    base on bool (default True), whether is weekend 
    or in between the week and if the moment in between 
    the week is 1 day apart or more"""
    if not moment_of_week: #day of querie is not in between the week
        day_separation=3 #querie to consult sunday (1), saturday (2) and friday (3)
        get_current_recent_date=get_monday_of_week() #get the date of monday from current week
    else:
        get_current_recent_date=datetime.now()
        print(get_current_recent_date)
    day_to_stop_search=get_current_recent_date-timedelta(days=1) #check from yesterday
    print(day_to_stop_search)
    day_before_yesterday=day_to_stop_search - timedelta(days=day_separation) #get yesterday date to check fraction of T1 and T2 y T3
    print(day_before_yesterday)
    return day_to_stop_search,day_before_yesterday

## Construct separation of days from last execution date to day to search in pre-usos
* store last execution date to compute separation of days

In [12]:
today=datetime.now() #get date_time from today
today_date=today.date() #get date
# Specify the file name
file_name = r".\date_time_last_exec.txt"
if today_date==get_monday_of_week(): #if it is monday
    not_weekend=False #moment of querie is weekend
    separation_of_days=None
else:
    not_weekend=True #moment of querie is not weekend
    # Read the content of the file and parse it as a datetime object
    with open(file_name, 'r') as file:
        last_execution = datetime.strptime(file.read().strip(), '%Y-%m-%d %H:%M:%S') #load last execution
        last_execution_date=last_execution.date()
        print(last_execution_date)
    separation_of_days=today_date-last_execution_date
    separation_of_days=separation_of_days.days if separation_of_days.days>0 else 365+separation_of_days.days #days separation to make queries in between week of more than 1 day
    print("separation",separation_of_days)
"""export last execution date into txt"""
# Write the date_time variable into the file
list_current_reports=find_files_by_format(Path.joinpath(path,"pptx_reports"),".pptx")
yesterday=today_date-timedelta(days=1)
most_recent_report_title_not_yesterday=True #flag to check yesterday report existance
for report_title in list_current_reports:
    if yesterday.strftime('%Y-%m-%d') in report_title:
        most_recent_report_title_not_yesterday=False
if most_recent_report_title_not_yesterday: #if a report with yesterday date is not already generated, update last execution
    with open(file_name, 'w') as file:
        print("no matches in reports, proceed to update last execution")
        file.write(today.strftime('%Y-%m-%d %H:%M:%S'))
day_to_stop_search_querie,day_before_yesterday_querie=date_limits(moment_of_week=not_weekend,day_separation=separation_of_days)
day_before_yesterday_querie=datetime.combine(day_before_yesterday_querie,shifts_hours.at[0,"initial_hour"])
day_to_stop_search_querie=datetime.combine(day_to_stop_search_querie,shifts_hours.at[2,"finalitation_hour"])
day_to_stop_search_querie,day_before_yesterday_querie

2025-02-06
separation 1
no matches in reports, proceed to update last execution
2025-02-07 09:45:10.656938
2025-02-06 09:45:10.656938
2025-02-05 09:45:10.656938


(datetime.datetime(2025, 2, 6, 22, 0), datetime.datetime(2025, 2, 5, 22, 0))

# Load pre-usos files

In [13]:
directory = Path(r".\pre_usos_actual") #get current work directory
matching_files = list(directory.glob("*Pre-Uso*.xlsx"))  # Busca archivos que tengan extensión .xlsx
print("Archivos encontrados:", matching_files)

Archivos encontrados: [WindowsPath('pre_usos_actual/Pre-Uso Astillado.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso BP2.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Formación y Prensa.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Lijadora.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Mantenimiento Mecánico.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Moduladora y Carpintería.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Planta Térmica.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Refinación.xlsx'), WindowsPath('pre_usos_actual/Pre-Uso Schelling.xlsx')]


In [14]:
dict_data_pointer={} #dict to store files as dfs
for i in matching_files: # Read the Excel file 
    file_path = str(i)  # Update this with the path
    try:
        df = pd.read_excel(file_path,sheet_name="Form1") #specific sheet call for english origin forms
    except:
        df = pd.read_excel(file_path)
    print(file_path)
    df_name=file_path.split(str(directory)+"\\")[-1].split(".")[0] #split str with "." char and take the file name
    dict_data_pointer[f"{df_name}"]=df #store filtered df in dictionary data pointer
print(list(dict_data_pointer.keys())) #see keys on dictionary to check callability

pre_usos_actual\Pre-Uso Astillado.xlsx
pre_usos_actual\Pre-Uso BP2.xlsx
pre_usos_actual\Pre-Uso Formación y Prensa.xlsx
pre_usos_actual\Pre-Uso Lijadora.xlsx
pre_usos_actual\Pre-Uso Mantenimiento Mecánico.xlsx
pre_usos_actual\Pre-Uso Moduladora y Carpintería.xlsx
pre_usos_actual\Pre-Uso Planta Térmica.xlsx
pre_usos_actual\Pre-Uso Refinación.xlsx
pre_usos_actual\Pre-Uso Schelling.xlsx
['Pre-Uso Astillado', 'Pre-Uso BP2', 'Pre-Uso Formación y Prensa', 'Pre-Uso Lijadora', 'Pre-Uso Mantenimiento Mecánico', 'Pre-Uso Moduladora y Carpintería', 'Pre-Uso Planta Térmica', 'Pre-Uso Refinación', 'Pre-Uso Schelling']


# Get forms done respect the shifts

# Make df to create excel with table of completions and specific rules of laboral days

In [15]:
day_difference=day_to_stop_search_querie-day_before_yesterday_querie
day_difference=day_difference.days
day_difference

1

## change creation of row_report with the reading from loading the actual workbook with the dates

In [16]:
df_pre_uso_completion_report=pd.read_excel(Path.joinpath(path,r"pre_uso_produccion_report_copia\Preusos Producción.xlsx"))
df_pre_uso_completion_report_cols=list(df_pre_uso_completion_report.columns)
array_data=[["","",""]*day_difference]*len(df_pre_uso_completion_report_cols[2:])
array_data=list(map(list, zip(*array_data))) #transpose list to match df input data format
print(array_data)
shifts=3
dates_of_interest= [ [date]*shifts for date in [day_to_stop_search_querie.date() - timedelta(days=x) for x in range(day_difference)] ]
dates_of_interest=list(itertools.chain.from_iterable(dates_of_interest))
if len(dates_of_interest)>1:
    dates_of_interest=sorted(dates_of_interest)
multi_index=[ dates_of_interest,["1","2","3"]*day_difference ]
print(multi_index)
df_check_completion=pd.DataFrame(data=array_data,index=multi_index,columns=df_pre_uso_completion_report_cols[2:])
df_check_completion.index.names =['Día',df_pre_uso_completion_report_cols[1]]
df_check_completion

[['', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '']]
[[datetime.date(2025, 2, 6), datetime.date(2025, 2, 6), datetime.date(2025, 2, 6)], ['1', '2', '3']]


Lijadora Schelling Moduladora y Carpintería BP2 Astillado  \
Día        Turno                                                             
2025-02-06 1                                                                 
           2                                                                 
           3                                                                 

                 Mantenimiento Mecánico Formación y Prensa Refinación  \
Día        Turno                                                        
2025-02-06 1                                                            
           2                                                            
           3                                                            

                 Planta Térmica  
Día        Turno                 
2025-02-06 1                     
           2                     
           3

## Function to check shifts

In [17]:
def shift_checker(date_time=None,system_to_test=None):
    system=system_to_test.lower() #convert system name to lowercase
    print(date_time)
    time_of_interest=date_time.time() #take the hh:mm:ss of entry in completion date_time
    initial_shift_hours=shifts_hours.loc[:,[shifts_hours_cols[0]]].values
    final_shift_hours=shifts_hours.loc[:,[shifts_hours_cols[1]]].values
    available_systems=[sys[0] for sys in shifts_hours.loc[:,[shifts_hours_cols[-1]]].values]
    for available_system in available_systems: #check which is the shift of the system
        if system in available_system:
            print(available_system)
            matching_shift=available_systems.index(available_system)+1
            print(matching_shift)
            if matching_shift<4: #system is from shift 1,2 or 3
                print(system," in ",matching_shift)
                if time_of_interest>=initial_shift_hours[0][0] or time_of_interest<final_shift_hours[0][0]: #check if it is shift 1
                    shift_result="T1"
                if time_of_interest>=initial_shift_hours[1][0] and time_of_interest<final_shift_hours[1][0]: #check if it is shift 2
                    shift_result="T2"
                if time_of_interest>=initial_shift_hours[2][0] and time_of_interest<final_shift_hours[2][0]: #check if it is shift 3
                    shift_result="T3"
            else: #system if from shift 4
                if time_of_interest>=initial_shift_hours[3][0] and time_of_interest<final_shift_hours[3][0]: #check if it is shift 3
                    shift_result="T4"
            return shift_result
        else:
            print("sistema no reconocido en {}".format(available_system))

## Apply check of shifts on dfs pre-usos

In [18]:
dict_result_shift_checker={}
for pre_uso in dict_data_pointer.keys(): #run throug all dfs of pre-usos
    df=dict_data_pointer[pre_uso] #select respective df
    df_columns=list(df.columns)
    df.loc[:,df_columns[1]] = pd.to_datetime(df.loc[:,df_columns[1]], format='%Y-%m-%d %H:%M:%S') # Convierte las columnas al formato datetime
    df_shift_dates=df.loc[( df[df_columns[1]]>=day_before_yesterday_querie )&( df[df_columns[1]]<=day_to_stop_search_querie )] #filter df to get completions of desired date
    df_shift_dates=df_shift_dates.sort_values(by=df_columns[1],ascending=False)
    df_shift_dates=df_shift_dates.fillna(value=" ")
    df_np_times=df_shift_dates.loc[:,df_columns[1]].values #get the dates of current df filtered
    """start shift check"""
    system_to_check_in=pre_uso.lower().split("pre-uso ")[-1]
    print(f"sistema a chequear: {system_to_check_in}")
    dict_result_shift_checker[system_to_check_in]=[]
    for date_np_time in df_np_times:
        date_timestamp=pd.Timestamp(date_np_time)
        print(f"fecha de diligenciamiento: {date_timestamp}")
        shift_checked=shift_checker(date_time=date_timestamp,system_to_test=system_to_check_in) #store shift
        print(f"turno asignado: {shift_checked}")
        dict_result_shift_checker[system_to_check_in].append((date_timestamp,shift_checked))
    dict_result_shift_checker[system_to_check_in]=sorted(dict_result_shift_checker[system_to_check_in])
dict_result_shift_checker

sistema a chequear: astillado
fecha de diligenciamiento: 2025-02-06 07:15:05
2025-02-06 07:15:05
['refinación', 'planta térmica', 'formación y prensa', 'bp2', 'astillado', 'mantenimiento mecánico']
1
astillado  in  1
turno asignado: T2
fecha de diligenciamiento: 2025-02-05 22:14:18
2025-02-05 22:14:18
['refinación', 'planta térmica', 'formación y prensa', 'bp2', 'astillado', 'mantenimiento mecánico']
1
astillado  in  1
turno asignado: T1
sistema a chequear: bp2
sistema a chequear: formación y prensa
fecha de diligenciamiento: 2025-02-06 14:17:57
2025-02-06 14:17:57
['refinación', 'planta térmica', 'formación y prensa', 'bp2', 'astillado', 'mantenimiento mecánico']
1
formación y prensa  in  1
turno asignado: T3
fecha de diligenciamiento: 2025-02-06 06:59:57
2025-02-06 06:59:57
['refinación', 'planta térmica', 'formación y prensa', 'bp2', 'astillado', 'mantenimiento mecánico']
1
formación y prensa  in  1
turno asignado: T2
fecha de diligenciamiento: 2025-02-06 06:26:45
2025-02-06 06:26:4

{'astillado': [(Timestamp('2025-02-05 22:14:18'), 'T1'),
  (Timestamp('2025-02-06 07:15:05'), 'T2')],
 'bp2': [],
 'formación y prensa': [(Timestamp('2025-02-05 22:04:56'), 'T1'),
  (Timestamp('2025-02-05 22:34:35'), 'T1'),
  (Timestamp('2025-02-06 06:26:45'), 'T2'),
  (Timestamp('2025-02-06 06:59:57'), 'T2'),
  (Timestamp('2025-02-06 14:17:57'), 'T3')],
 'lijadora': [(Timestamp('2025-02-06 06:25:46'), 'T2'),
  (Timestamp('2025-02-06 14:12:31'), 'T3')],
 'mantenimiento mecánico': [(Timestamp('2025-02-06 06:02:55'), 'T2')],
 'moduladora y carpintería': [(Timestamp('2025-02-06 13:23:16'), 'T2')],
 'planta térmica': [(Timestamp('2025-02-05 22:31:54'), 'T1'),
  (Timestamp('2025-02-06 07:00:17'), 'T2'),
  (Timestamp('2025-02-06 13:21:54'), 'T2'),
  (Timestamp('2025-02-06 14:17:39'), 'T3')],
 'refinación': [(Timestamp('2025-02-05 22:34:15'), 'T1'),
  (Timestamp('2025-02-05 23:27:23'), 'T1'),
  (Timestamp('2025-02-06 07:00:32'), 'T2'),
  (Timestamp('2025-02-06 08:03:13'), 'T2'),
  (Timestamp(

## asign values on df of completion

In [19]:
df_check_completion_cols=df_check_completion.columns
for (key_shift_checker,values_shift_checker) in dict_result_shift_checker.items():
    print(key_shift_checker,values_shift_checker)
    for current_col_check_completion in df_check_completion_cols:
        if key_shift_checker in current_col_check_completion.lower():
            print(f"sistema {key_shift_checker} in {current_col_check_completion}")
            for date_time,shift in values_shift_checker: #begin asignation by dates and shifts
                current_date_of_shift_checker=date_time.date()
                idx_shift=shift[-1]
                print(current_date_of_shift_checker)
                if current_date_of_shift_checker not in dates_of_interest:
                    current_date_of_shift_checker=min(dates_of_interest) #if date not in dates_of_interest, take the youngest date
                print(idx_shift)
                df_check_completion.loc[(current_date_of_shift_checker, idx_shift), current_col_check_completion]="OK" #asignation
df_check_completion

astillado [(Timestamp('2025-02-05 22:14:18'), 'T1'), (Timestamp('2025-02-06 07:15:05'), 'T2')]
sistema astillado in Astillado
2025-02-05
1
2025-02-06
2
bp2 []
sistema bp2 in BP2
formación y prensa [(Timestamp('2025-02-05 22:04:56'), 'T1'), (Timestamp('2025-02-05 22:34:35'), 'T1'), (Timestamp('2025-02-06 06:26:45'), 'T2'), (Timestamp('2025-02-06 06:59:57'), 'T2'), (Timestamp('2025-02-06 14:17:57'), 'T3')]
sistema formación y prensa in Formación y Prensa
2025-02-05
1
2025-02-05
1
2025-02-06
2
2025-02-06
2
2025-02-06
3
lijadora [(Timestamp('2025-02-06 06:25:46'), 'T2'), (Timestamp('2025-02-06 14:12:31'), 'T3')]
sistema lijadora in Lijadora
2025-02-06
2
2025-02-06
3
mantenimiento mecánico [(Timestamp('2025-02-06 06:02:55'), 'T2')]
sistema mantenimiento mecánico in Mantenimiento Mecánico
2025-02-06
2
moduladora y carpintería [(Timestamp('2025-02-06 13:23:16'), 'T2')]
sistema moduladora y carpintería in Moduladora y Carpintería
2025-02-06
2
planta térmica [(Timestamp('2025-02-05 22:31:54'), 

Lijadora Schelling Moduladora y Carpintería BP2 Astillado  \
Día        Turno                                                             
2025-02-06 1                                                            OK   
           2           OK        OK                       OK            OK   
           3           OK        OK                                          

                 Mantenimiento Mecánico Formación y Prensa Refinación  \
Día        Turno                                                        
2025-02-06 1                                            OK         OK   
           2                         OK                 OK         OK   
           3                                            OK         OK   

                 Planta Térmica  
Día        Turno                 
2025-02-06 1                 OK  
           2                 OK  
           3                 OK

## export completion df to excel

In [20]:
path_to_export=Path.joinpath(path,rf'row_date_report\{day_to_stop_search_querie.date()}.xlsx')
print(path_to_export)
df_check_completion.to_excel(path_to_export)

C:\Users\jdbustamante\OneDrive - Duratex SA\reports_pre-usos_diligenciamiento_no_cumple\source_and_return_data\row_date_report\2025-02-06.xlsx


# Filter dfs of preusos on one cell to get "No Cumple"

In [21]:
dict_data_no_cumple={}
for pre_uso_key in list(dict_data_pointer.keys()):
    print(f"{pre_uso_key} to search in No Cumple")
    """select df"""
    df=dict_data_pointer[pre_uso_key]
    """save columns names"""
    df_columns=list(df.columns)
    # Convierte las columnas al formato datetime
    df.loc[:,df_columns[1]] = pd.to_datetime(df.loc[:,df_columns[1]], format='%Y-%m-%d %H:%M:%S')
    """filter dates to check completion"""
    df_shift_dates=df.loc[( df[df_columns[1]]>=day_before_yesterday_querie )&( df[df_columns[1]]<=day_to_stop_search_querie )] #get completion dates
    df_shift_dates=df_shift_dates.sort_values(by=df_columns[1],ascending=False)
    df_shift_dates=df_shift_dates.fillna(value=" ")
    
    """create df to check observations on No Cumple entries"""
    df_to_concatenate=pd.DataFrame() #df to begin concatenation
    for col_name in df_columns[8:]:
        print(f"has {col_name} No Cumple?")
        df_col_name_querie=df_shift_dates.loc[ (df_shift_dates[col_name].astype("str").str.contains("No Cumple"))|(df_shift_dates[col_name].astype("str").str.contains("No cumple")) ] #--->reformat to make cleaner solution
        if len(df_col_name_querie)>0: #"No Cumple" is found in current col
            print(f"{col_name} has No Cumple")
            df_with_concatenate=df_col_name_querie.loc[:,[col_name]]#df_col_name_querie.loc[:,[df_columns[1],df_columns[-1],col_name]]
            df_to_concatenate=pd.concat([df_to_concatenate, df_with_concatenate],axis=1, sort=False)
    indexes_dates_observs=list(df_to_concatenate.index)
    df_to_concatenate=pd.concat([df_shift_dates.loc[indexes_dates_observs,[df_columns[-1],df_columns[1] ]],df_to_concatenate],axis=1, sort=False)
    df_to_concatenate=df_to_concatenate.fillna(value=" ")#sort_values(by=df_columns[1])
    if len(df_to_concatenate)>0: #only take the queries that are not empty
        dict_data_no_cumple[pre_uso_key]=df_to_concatenate

Pre-Uso Astillado to search in No Cumple
has ¿El aire acondicionado esta operativo? No Cumple?
has ¿Las cajas de control, empaques, cableado y tomas eléctricos se encuentran cerrados y en buen estado? No Cumple?
¿Las cajas de control, empaques, cableado y tomas eléctricos se encuentran cerrados y en buen estado? has No Cumple
has ¿Están ubicadas y funcionales las 6 guardas de seguridad del tambor descortezador? No Cumple?
¿Están ubicadas y funcionales las 6 guardas de seguridad del tambor descortezador? has No Cumple
has ¿Los interruptores de paro de emergencia están funcionales? No Cumple?
has ¿La unidad hidráulica y accesorios presentan algún tipo de fuga? No Cumple?
has ¿Las escaleras, pasamanos y paros de cuerdas están funcionales y libre de obstrucciones? No Cumple?
¿Las escaleras, pasamanos y paros de cuerdas están funcionales y libre de obstrucciones? has No Cumple
has ¿Las barreras perimetrales de la mesa transversal están ubicadas y funcionales? No Cumple?
¿Las barreras perime

# load data from SSs to make follow-up slides

In [54]:
follow_up_format=".xlsx"
follow_up_path=Path(r".\service_requests_maintenance_mantum")
list_follow_ups=[]
list_f_ups_cmms=[]
for file_path in rp.find_files_by_format(follow_up_path,follow_up_format):
    if "seguimiento" in file_path:
        list_follow_ups.append(file_path)
    elif "combined_data" not in file_path: #take export from cmms
        list_f_ups_cmms.append(file_path)
dataframes=[]
# Loop through each file
for follow_ups_cmms in list_f_ups_cmms:
    # Load the Excel file, skipping the first row
    df = pd.read_excel(follow_ups_cmms, skiprows=1)
    # Append the DataFrame to the list
    dataframes.append(df)
# Concatenate all DataFrames into one
df_to_take_ots = pd.concat(dataframes, ignore_index=True)
df_to_take_ots=df_to_take_ots.astype("str")
df_to_take_ots=df_to_take_ots.drop_duplicates()
df_to_take_ots.to_excel(r".\service_requests_maintenance_mantum\combined_data.xlsx", index=False) #export merged df
selected_df_to_take_ots_cols=list(df_to_take_ots.columns)
dict_data_SSs={} #dict to store dfs of follo-ups
string_from_float=lambda x:str(int(float(x)))
for follow_up in list_follow_ups:
    df=pd.read_excel(follow_up, sheet_name="ss")
    len_df=len(df)
    df.index=range(len_df) #set new index naive
    #df["descartable"]=["no"]*len_df #insert OT col
    df["OT"]=["nan"]*len_df #insert OT col
    selected_df_cols=list(df.columns) #take all cols, except last: its just non relevant text
    df=df.astype("str")
    #take OTs from pending services is available
    for ss_code in df.loc[:,selected_df_cols[2]].values: #take ss code from seguimiento
        if "nan" in ss_code:
            continue
        df_matches=df_to_take_ots.loc[df_to_take_ots[selected_df_to_take_ots_cols[0]].str.contains(string_from_float(ss_code))] #take data which contains ss code
        print("coincidencias: ",df_matches)
        if df_matches.empty: #ss doesnt have ot
            continue
        ot_index_in_df_matches=list(df_matches.index)[0]
        print("valor indice: ",ot_index_in_df_matches)
        print("tamaño df: ",len(df_matches))
        if len(df_matches)<1:
            continue
        ot_value=df_matches.at[ot_index_in_df_matches,selected_df_to_take_ots_cols[-1]] #assign ot code#df_matches.loc[:,selected_df_to_take_ots_cols[-1]].values
        print("valor ot: ",ot_value)
        ot_index_in_df=df[df[selected_df_cols[2]].str.contains(ss_code)].index
        df.loc[ot_index_in_df,selected_df_cols[-1]]=ot_value#df_matches.at[ot_index_in_df_matches,selected_df_to_take_ots_cols[0]] #assign ot code
    df=df[df[selected_df_cols[2]].notna()].loc[:,selected_df_cols[1:]] #select required cols
    df[selected_df_cols[2]]=df.loc[:,selected_df_cols[2]].apply(lambda x:x.split(".")[0]) #remove zeros from ss codes
    df=df.replace("nan", np.nan) #replace str nan with foundable nans
    df=df.dropna() #delete nans
    dict_data_SSs[follow_up.split("SSs_")[-1].split(follow_up_format)[0]]=df.loc[:,selected_df_cols[1:]]
dict_data_SSs

coincidencias:        Código Actividad Entidad Tipo Especialidad Frecuencia  Estado  \
10043  13042       nan     nan  nan          nan        nan  Creada   

      610 Lijadora 640 Schelling       F. Creación    Fecha Esperada  \
10043          nan           nan  2025-01-21 11:29  2025-02-20 11:29   

                                        Entidad Asociada  \
10043  030.044.001.006 | Sistema Estructural Banda Tr...   

                        Solicitante  \
10043  JOSE DAVID BUSTAMANTE SIERRA   

                                             Descripción Prioridad  \
10043  varias canaletas contienen astilla y estan des...     Media   

                   Tipo S.S.                       Área    %  \
10043  Correctivo Programado  MANTENIMIENTO ELECTRÓNICO  0.0   

                    ANS  O.T  
10043  2025-05-10 11:29  nan  
valor indice:  10043
tamaño df:  1
valor ot:  nan
coincidencias:        Código Actividad Entidad Tipo Especialidad Frecuencia  Estado  \
10032  13041       nan     

C:\Users\jdbustamante\AppData\Local\Temp\ipykernel_24808\1622722123.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df=df.replace("nan", np.nan) #replace str nan with foundable nans


{'astillado':                                              hallazgo     SS     responsable  \
 8   la pasarela al costado de transporte de tronco...  13045  Mntto mecánico   
 12  guarda incompleta ya que no cubren toda la cad...  13049  Mntto mecánico   
 15  el pin queda inseguro cuando se va la Luz debi...  13069  Mntto mecánico   
 20  Durante inspección se reporta condición de pun...  13023  Mntto mecánico   
 21  hueco en el suelo donde se posiciona el cargad...  12930  Mntto Locativo   
 31           pin de bloqueo de rotor con fuga de aire  13077  Mntto mecánico   
 33  afiladora por cambio de mangueras de agua y de...  13071  Mntto mecánico   
 
               OT  
 8    085578 | 0%  
 12   085199 | 0%  
 15   085193 | 0%  
 20   084870 | 0%  
 21   084801 | 0%  
 31   085191 | 0%  
 33   085192 | 0%  ,
 'lijadora': Empty DataFrame
 Columns: [hallazgo, SS, responsable, OT]
 Index: []}

# test matches rutinarias
* DONE:
   * cotejo de actividades rutinarias entre ams de mantum, solicitudes de servicio combinadas y solicitudes relacionadas con cargo
* TODO:
   * inserción de flag indicando si follow-up es descartable o no en columna respectiva

In [55]:
# Cargar los archivos en DataFrames (ajustar las rutas según corresponda)
source = pd.read_excel(r".\service_requests_maintenance_mantum\combined_data.xlsx")  # Reemplazar con la ruta del archivo de Source
mantum_am = pd.read_excel(r".\service_requests_maintenance_mantum\listaAMsActivas.xlsx",skiprows=1)  # Reemplazar con la ruta del archivo de Mantum_AM
follow_up_sss = pd.read_excel(r".\service_requests_maintenance_mantum\ListaSolicitudServicioPendientesMantum\ListaSolicitudServicioSolicitadas.xlsx",skiprows=1)  # Archivo de Follow_Up_SSS
"""inicio de coincidencias"""
# Buscar la columna con el nombre que contenga "Entidad Asociada" en source
source_column_name = next((col for col in source.columns if "Entidad Asociada" in col), None)

# Buscar la columna con el nombre que contenga "Entidad" en mantum_am
mantum_column_name = next((col for col in mantum_am.columns if "Entidad" in col), None)
mantum_type_col = next((col for col in mantum_am.columns if "Tipo" in col), None)
#buscar columna de ubicación de entidad en follow_up
follow_up_column_name = next((col for col in follow_up_sss.columns if "Entidad Asociada" in col), None)
# Verificar que todas las columnas fueron encontradas
if None in [source_column_name, mantum_column_name, follow_up_column_name]:
    print("Error: No se encontraron las columnas necesarias en los archivos.")
else:
    # Iterar sobre cada entidad en Source
    for index, entidad_source in source[source_column_name].items():
        # Verificar si la entidad de Source está en Mantum_AM
        matching_rows_mantum = mantum_am[mantum_am[mantum_column_name] == entidad_source]

        # Si hay coincidencias en Mantum_AM, verificar "Tipo" y la ubicación en Follow_Up_SSS
        for mantum_index in matching_rows_mantum.index:
            if matching_rows_mantum.loc[mantum_index, "Tipo"] == "Rutinaria":
                # Verificar si la entidad está en Follow_Up_SSS por ubicación
                matching_rows_follow_up = follow_up_sss[follow_up_sss[follow_up_column_name] == entidad_source]

                for follow_up_index in matching_rows_follow_up.index:
                    print(f"Coincidencia en índice {mantum_index} de Mantum_AM y en índice {follow_up_index} de Follow_Up_SSS para '{entidad_source}'.")

Coincidencia en índice 791 de Mantum_AM y en índice 7 de Follow_Up_SSS para '030.041.001 | Unidad Hidraulica de Descortezador'.
Coincidencia en índice 792 de Mantum_AM y en índice 7 de Follow_Up_SSS para '030.041.001 | Unidad Hidraulica de Descortezador'.
Coincidencia en índice 794 de Mantum_AM y en índice 7 de Follow_Up_SSS para '030.041.001 | Unidad Hidraulica de Descortezador'.
Coincidencia en índice 208 de Mantum_AM y en índice 4 de Follow_Up_SSS para '050.051 | Banda Alimentacion Chipper'.
Coincidencia en índice 209 de Mantum_AM y en índice 4 de Follow_Up_SSS para '050.051 | Banda Alimentacion Chipper'.
Coincidencia en índice 231 de Mantum_AM y en índice 6 de Follow_Up_SSS para '050.G01 | Gabinete de operación de Astillado 100-LCS-01'.
Coincidencia en índice 791 de Mantum_AM y en índice 7 de Follow_Up_SSS para '030.041.001 | Unidad Hidraulica de Descortezador'.
Coincidencia en índice 792 de Mantum_AM y en índice 7 de Follow_Up_SSS para '030.041.001 | Unidad Hidraulica de Descortez

# Construction of data to put on presentation

# Write in presentation
## TODO
* If size of observations is adequate, put no completion entries, questions label and observations in the same slide
* Table must fit on one slide without exceding the size of the slide

In [56]:
def iter_cells(table):
    """
    to change font of text in table
    """
    for row in table.rows:
        for cell in row.cells:
            yield cell

In [57]:
prs = Presentation(r".\pptx_reports\Preusos Producción y MMTTO_template.pptx")
"""make title slide to follow-ups"""
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]
title.text = "Pre-Usos BAR"
subtitle_text="Diligenciamientos"
subtitle.text = " "+subtitle_text
"""make next slides to inform about pre-usos"""
"""make completion report slide"""
title_no_content_slide_layout=prs.slide_layouts[5]
slide = prs.slides.add_slide(title_no_content_slide_layout)
title = slide.shapes.title
title.text="Pre-Usos periodo {}, {}".format(day_before_yesterday_querie.date(),day_to_stop_search_querie.date()) #datetime.now().date()
title_para = slide.shapes.title.text_frame.paragraphs[0]
font_title_slide=20 #font to title of slides
title_para.font.size = Pt(font_title_slide) #set title size
"""make slides to report about No Cumple"""
x, y, cx, cy = Inches(0.1), Inches(0.4), Inches(5), Inches(5)
factorx=0.4
factory=0.25
offsets=[cx*(1-factorx),cy*(1+factory)] #to adjust textbox @right of table
simple_content_slide_layout=prs.slide_layouts[1]
font_title_slide=20 #font to title of slides
max_font_to_fit_text=15 #value for max font size when fitting text
font_table_text=12 #font size to be set on tables
for pre_uso_name in dict_data_no_cumple.keys(): #take each df with no cumple entries
    slide=prs.slides.add_slide(title_no_content_slide_layout) #slide to add dates and no completion items
    title = slide.shapes.title
    title.text=pre_uso_name

    title_para = slide.shapes.title.text_frame.paragraphs[0]
    title_para.font.size = Pt(font_title_slide) #set title size
    
    df=dict_data_no_cumple[pre_uso_name].T
    table_rows,table_cols=df.shape
    #print(table_rows,table_cols)
    """insert text box label to identify items with questions"""
    txBox=slide.shapes.add_textbox(x+cx, Inches(0.2), Inches(8), offsets[1])
    tf = txBox.text_frame
    tf.word_wrap = True
    #tf.autofit_text()
    p = tf.add_paragraph()
    text_container = "" #var to store questions of respective pre-uso
    #p.font.size = Pt(12)
    """make table of no completions"""
    shape = slide.shapes.add_table(table_rows-1, table_cols+1, x, y, cx, cy) #table geometry
    table=shape.table
    cell = table.cell(0, 0)
    cell.text="Item"
    df_col_index=list(df.index)[1:] #name of current index which were columns before
    print(df_col_index)
    df_index_col=list(df.columns) #name of current cols which were indexes before
    for idx_col in range(table_cols): #write on header cells of table
        print("indice de columna en tabla: ",idx_col)
        cell = table.cell(0, idx_col+1)
        cell.text="{}".format(df.at[ df_col_index[0],df_index_col[idx_col] ])
     #write on first column to put questions with no completion
    for idx_row in range(table_rows-2):
        print("indice de fila en tabla: ",idx_row)
        cell = table.cell(idx_row+1, 0)
        print(df_col_index[idx_row+1])
        cell.text="{}".format(idx_row+1) #df_col_index[idx_row+1]
        if idx_row<1:
            text_container+= f"->Pregunta {idx_row+1}: "+df_col_index[idx_row+1] #insert text to label_id_questions text box without next line insertion
        else:
            text_container+= f"\n->Pregunta {idx_row+1}: "+df_col_index[idx_row+1] #insert text to label_id_questions text box 
    p.text=text_container #put text on textbox
    tf.fit_text(max_size=max_font_to_fit_text) #adjust text to fit int
    #insert respective data
    for idx_row in range(table_rows-2):
        for idx_col in range(table_cols):
            print("celda value:",idx_row+1,idx_col)
            cell = table.cell(idx_row+1, idx_col+1)
            cell.text="{}".format(df.at[ df_col_index[idx_row+1],df_index_col[idx_col] ]) #insert df value
    """to change table text font"""
    for cell in iter_cells(table):
        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(font_table_text)
    """to add observations"""
    string_to_add_observations=""
    flag_make_observations_slide=False
    for idx_col in range(table_cols):
    #put date of observation and observation content
        observation=df.at[ list(df.index)[0],df_index_col[idx_col] ]
        min_lenght_observation=2 #min lenght of observation to be inserted on slides report
        if len(observation)<min_lenght_observation: #if observation lenght is minor to 1 chars, continue
            continue
        else:
            flag_make_observations_slide=True
            if idx_col<1:
                string_to_add_observations+=f"Observación {df.at[ df_col_index[0],df_index_col[idx_col] ]}: {observation}" #insert 1st observation without next line insertion
            else:
                string_to_add_observations+=f"\nObservación {df.at[ df_col_index[0],df_index_col[idx_col] ]}: {observation}" #insert 1st observation without next line insertion            
    if flag_make_observations_slide:
        slide=prs.slides.add_slide(simple_content_slide_layout) #slide to add observations
        title = slide.shapes.title
        title.text="{}-{}".format(pre_uso_name,"Observaciones")
    
        title_para = slide.shapes.title.text_frame.paragraphs[0]
        title_para.font.size = Pt(font_title_slide) #set title size
        """insert content on text box place holder"""
        content_tf=slide.placeholders[1]
        format_text_props=content_tf.text_frame
        format_text_props.word_wrap=True
        content_tf.text=string_to_add_observations
        format_text_props.fit_text(max_size=12) #adjust text to fit int

"""to add SSs """
"""make title slide to follow-ups"""
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]
title.text = "Pre-Usos BAR"
subtitle_f_up_text=""
for key_f_up in dict_data_SSs.keys():
    subtitle_f_up_text+=f"\n{key_f_up}"
subtitle.text = "Seguimiento de SSs: "+subtitle_f_up_text
for follow_up_name in dict_data_SSs.keys(): #take each df with follow_up entries
    print(follow_up_name)
    slide=prs.slides.add_slide(title_no_content_slide_layout) #slide to add dates and no completion items
    title = slide.shapes.title
    title.text=follow_up_name+" SSs abiertas"
    title_para = slide.shapes.title.text_frame.paragraphs[0]
    title_para.font.size = Pt(font_title_slide) #set title size
    df=dict_data_SSs[follow_up_name]
    table_rows,table_cols=df.shape
    """make table of follow-ups"""
    shape = slide.shapes.add_table(table_rows+1, table_cols, Inches(0), Inches(0.5), Inches(13.3), Inches(6)) #table geometry
    table=shape.table
    for idx_col,col in enumerate(df.columns):
        cell = table.cell(0, idx_col)
        cell.text="{}".format(col) #insert df value
    for idx_row,idx_df in enumerate(df.index):
        print(f"row {idx_row}")
        for idx_col,col in enumerate(df.columns):
            print(f"col {idx_col}")
            cell = table.cell(idx_row+1, idx_col)
            if idx_col==0:
                cell.text="{}".format(df.at[ idx_df, col]) #insert hallazgo
                print(cell.text)
            else:
                cell.text="{}".format(df.at[ idx_df, col]) #insert rest of values
    """to change table text font"""
    for cell in iter_cells(table):
        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(12)

presentation_date=day_to_stop_search_querie.date()#.strftime('%d-%m-%Y')
print(presentation_date)
year_folder=rf"\year_{presentation_date.year}"
month_folder=rf"\month_{day_to_stop_search_querie.month}"
directory=r"pptx_reports"
directory_to_save = Path.joinpath(path,str(directory)+year_folder+month_folder) # get directory to save plot
directory_to_save.mkdir(exist_ok=True)
final_directory=str(directory_to_save)+rf"\Preusos Producción y MMTTO_{presentation_date}.pptx"
prs.save(final_directory)
test_directory=Path.joinpath(path,str(directory)+r'\test\test.pptx')
prs.save(test_directory)

['Hora de inicio', '¿Las cajas de control, empaques, cableado y tomas eléctricos se encuentran cerrados y en buen estado?', '¿Están ubicadas y funcionales las 6 guardas de seguridad del tambor descortezador?', '¿Las escaleras, pasamanos y paros de cuerdas están funcionales y libre de obstrucciones?', '¿Las barreras perimetrales de la mesa transversal están ubicadas y funcionales?', '¿Están ubicadas y funcionales las guardas de seguridad de las bandas de descarga de corteza, transportadora de tronco y\xa0alimentación al chipper?', '¿Las escaleras, pasamanos y senderos peatonales están libre de obstrucciones y en buen estado?', '¿Las barreras perimetrales de las poleas de las poleas del Chipper están ubicadas y funcionales?', '¿Las guardas de seguridad de los rodillos dentados están ubicadas y funcionales (8 unidades)?', '¿Las cajas eléctricas, el pin de bloqueo y los tornillos de ajustes del Chipper están ajustados?', '¿La guarda de seguridad de los tornillos gemelos están ubicadas y fu

In [58]:
# Ruta de la presentación (modifica según tu archivo)
for pptx_path in [final_directory,test_directory]:
    
    # Abrir la presentación
    prs = Presentation(pptx_path)
    
    # Obtener la primera diapositiva
    first_slide = prs.slides[0]
    
    # Obtener la fecha actual en formato día/mes/año
    today_date = last_execution_date.strftime("%d/%m/%Y") #datetime.today().strftime("%d/%m/%Y")
    
    # Verificar si la diapositiva tiene un marcador de posición para el subtítulo
    if len(first_slide.placeholders) > 1:  
        subtitle_placeholder = first_slide.placeholders[1]  # Placeholder del subtítulo
        subtitle_placeholder.text = today_date  # Insertar la fecha
    else:
        print("Advertencia: La primera diapositiva no tiene un marcador de subtítulo.")
    
    # Guardar la presentación con los cambios
    prs.save(pptx_path)
    print(f"Fecha '{today_date}' insertada en el subtítulo de la primera diapositiva.")

Fecha '06/02/2025' insertada en el subtítulo de la primera diapositiva.
Fecha '06/02/2025' insertada en el subtítulo de la primera diapositiva.
